In [24]:
import os, joblib
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, Bidirectional, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
import torch
import torch.distributed as dist 
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# from google.colab import drive
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense
import numpy as np
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [25]:
# Define Data directory
DATA_DIR = '../data/'

# list of poet names
# Removed the 5 lowest poets
poets = ["abbasinYousuf", "ajmalKhattak", "allamaAbdulHai", "ghaniKhan", "hamzaBaba", "javedAhmedzai", "karanKhan", "khatirAfridi", "khushalKhanKhattak", "matiullahTurab", "mumtazOrakazi", "munirJan", "naeemAhmed", "rabiaMumtaz", "rahmanBaba", "rehmatShah", "sahibShahSabir", "shabbirKhanDurrani", "shakirOrakzai", "shoaibKhanKhattak"]

poet_labels = {poet: i for i, poet in enumerate(poets)}
print(poet_labels)

{'abbasinYousuf': 0, 'ajmalKhattak': 1, 'allamaAbdulHai': 2, 'ghaniKhan': 3, 'hamzaBaba': 4, 'javedAhmedzai': 5, 'karanKhan': 6, 'khatirAfridi': 7, 'khushalKhanKhattak': 8, 'matiullahTurab': 9, 'mumtazOrakazi': 10, 'munirJan': 11, 'naeemAhmed': 12, 'rabiaMumtaz': 13, 'rahmanBaba': 14, 'rehmatShah': 15, 'sahibShahSabir': 16, 'shabbirKhanDurrani': 17, 'shakirOrakzai': 18, 'shoaibKhanKhattak': 19}


In [26]:
# Load and Label the Data
def load_and_label(data_dir, poets, poet_labels):
    data = []
    labels = []

    for poet in poets:
        poet_dir = os.path.join(data_dir, poet)
        file_path = os.path.join(poet_dir, f'{poet}.txt')

        # Check if the file exists
        if not os.path.exists(file_path):
            print(f'{file_path} does not exist')
            continue

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            data.extend(lines)
            labels.extend([poet_labels[poet]] * len(lines))

    df = pd.DataFrame({'text': data, 'label': labels})
    return data, labels, df

In [27]:
data, labels, df = load_and_label(DATA_DIR, poets, poet_labels)
df.describe()

,label
count,54620.000000
mean,8.625265
std,5.352617
min,0.000000
25%,4.000000
50%,8.000000
75%,14.000000
max,19.000000


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
poems_train, poems_temp, lables_train, labels_temp = train_test_split(data, labels, test_size=0.2, random_state=42)
poems_val, poems_test, labels_val, labels_test = train_test_split(poems_temp, labels_temp, test_size=0.5, random_state=42)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# padding token
tokenizer.pad_token = tokenizer.eos_token

### Llama

In [18]:
# Function to save embeddings
def save_embeddings(texts, prefix, model, tokenizer, device, max_length=80):
    for i, text in enumerate(texts):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        
        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)
            # Use the mean of the last hidden state as the embedding
            embedding = outputs.hidden_states[-1].mean(dim=1).to(device)
        
        torch.save(embedding, f'./models_llama/{prefix}_embedding_{i}.pt')
        if i % 100 == 0:
            print(f"{prefix.capitalize()} Embedding {i} saved.")


In [19]:
# Save embeddings for train, test and validation
save_embeddings(poems_train, 'train', model, tokenizer, device)
save_embeddings(poems_val, 'val', model, tokenizer, device)
save_embeddings(poems_test, 'test', model, tokenizer, device)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
class_labels = sorted(list(set(lables_train)))

def load_embeddings(prefix, num_embeddings):
    embeddings = []
    for i in range(num_embeddings):
        embedding = torch.load(f'./models_llama/{prefix}_embedding_{i}.pt')
        embeddings.append(embedding)
    return torch.stack(embeddings)

In [ ]:
train_embeddings = load_embeddings('train', len(poems_train))
val_embeddings = load_embeddings('val', len(poems_val))
test_embeddings = load_embeddings('test', len(poems_test))

In [ ]:
train_labels = torch.tensor(lables_train)
val_labels = torch.tensor(labels_val)
test_labels = torch.tensor(labels_test)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
# encoded_labels = label_encoder.fit_transform(lables_train)

In [ ]:
train_embeddings_np = train_embeddings.cpu().detach().numpy()
val_embeddings_np = val_embeddings.cpu().detach().numpy()
test_embeddings_np = test_embeddings.cpu().detach().numpy()
labels_train_np = np.array(lables_train)
labels_val_np = np.array(labels_val)
labels_test_np = np.array(labels_test)

In [ ]:
train_embeddings_np.shape, val_embeddings_np.shape, test_embeddings_np.shape, labels_train_np.shape, labels_val_np.shape, labels_test_np.shape

In [ ]:
#BERT WITH BERT

class PoemClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(PoemClassifier, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten along the sequence length dimension
        x = F.relu(x)
        x = self.fc(x)
        return x

In [ ]:
# set the input size based on the size of the embeddings
input_size = train_embeddings.size(2)

# initialize the classifier and move it to the device
classifier = PoemClassifier(input_size, len(labels)).to(device)
# classifier = PoemClassifier(input_size, len(class_labels)).to(device)
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    classifier = nn.DataParallel(classifier)
else:
    print("Using single GPU")

# Define the optimizer and loss function
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Create dataloader for training to iterate over the embeddings and labels in batches
train_dataset = TensorDataset(train_embeddings, train_labels)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
# # set the input size based on the size of the embeddings
# input_size = train_embeddings.size(2)

# # initialize the classifier and move it to the device
# classifier = PoemClassifier(input_size, len(labels)).to(device)
# # classifier = PoemClassifier(input_size, len(class_labels)).to(device)

# # Define the optimizer and loss function
# optimizer = optim.Adam(classifier.parameters(), lr=0.001)
# criterion = nn.CrossEntropyLoss()

# # Create dataloader for training to iterate over the embeddings and labels in batches
# train_dataset = TensorDataset(train_embeddings, train_labels)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
val_accuracies = []
train_accuracies = []
best_val_loss = float('inf')
patience = 5  # You can adjust this based on your needs
counter = 0
train_losses = []  # Track training loss
val_losses = []

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(TensorDataset(val_embeddings, val_labels), batch_size = 8)
accumalation_steps = 4

In [ ]:
train_loader, val_loader

In [ ]:
# Train Loop with early stopping
num_epochs = 10
for epoch in range(num_epochs):
    classifier.train()
    epoch_loss = 0
    optimizer.zero_grad()

    for i, (batch_embeddings, batch_labels) in enumerate(train_loader):
        if i % 100 == 0: print(f"Iteration {i}/{len(train_loader)} of train loader")
        batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

        # optimizer.zero_grad()
        output = classifier(batch_embeddings)
        loss = criterion(output, batch_labels)
        # loss.backward()
        loss = loss / accumalation_steps
        loss.backward()
        # optimizer.step()

        if (i + 1) % accumalation_steps == 0 or (i + 1) == len(train_loader):
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item() * accumalation_steps

    avg_epoch_loss = epoch_loss / len(train_loader.dataset)
    train_losses.append(avg_epoch_loss)

    print(f"Epoch {epoch + 1}/{num_epochs} - Training and Validation Accuracy:")

    # Training accuracy and data in batches to avoid over memory consumption
    classifier.eval()
    train_correct = 0
    train_total = 0
    with torch.no_grad():
        for batch_embeddings, batch_labels in train_loader:
            batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)
            train_outputs = classifier(batch_embeddings)
            train_probabilities = F.softmax(train_outputs, dim=1)
            train_predicted_labels = torch.argmax(train_probabilities, dim=1)
            train_correct += (train_predicted_labels == batch_labels).sum().item()
            train_total += batch_labels.size(0)
    train_accuracy = train_correct / train_total
    train_accuracies.append(train_accuracy)


    # Validation accuracy and loss in batches to avoid over memory consumption
    val_correct = 0
    val_total = 0
    val_loss_total = 0
    with torch.no_grad():
        for val_batch_embeddings, val_batch_labels in DataLoader(TensorDataset(val_embeddings, val_labels), batch_size=8):
            val_batch_embeddings, val_batch_labels = val_batch_embeddings.to(device), val_batch_labels.to(device)
            val_outputs = classifier(val_batch_embeddings)
            val_loss = criterion(val_outputs, val_batch_labels)
            val_loss_total += val_loss.item()

            val_probabilities = F.softmax(val_outputs, dim=1)
            val_predicted_labels = torch.argmax(val_probabilities, dim=1)
            val_correct += (val_predicted_labels == val_batch_labels).sum().item()
            val_total += val_batch_labels.size(0)

        val_accuracy = val_correct / val_total
        val_losses.append(val_loss_total / len(val_loader))

    # Print current status
    print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_epoch_loss:.4f}, Val Loss: {val_loss_total / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.4f}")

    # Early stopping check
    if val_loss_total < best_val_loss:
        best_val_loss = val_loss_total
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early Stopping!")
            break

In [ ]:
# # Train Loop with early stopping
# num_epochs = 100
# for epoch in range(num_epochs):
#     classifier.train()
#     epoch_loss = 0

#     for batch_embeddings, batch_labels in train_loader:
#         batch_embeddings, batch_labels = batch_embeddings.to(device), batch_labels.to(device)

#         optimizer.zero_grad()
#         output = classifier(batch_embeddings)
#         loss = criterion(output, batch_labels)
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#     avg_epoch_loss = epoch_loss / len(train_loader.dataset)
#     train_losses.append(avg_epoch_loss)

#     # Training acuracy
#     with torch.no_grad():
#         classifier.eval()
#         train_outputs = classifier(train_embeddings.to(device))
#         train_probabilities = F.softmax(train_outputs, dim=1)
#         train_predicted_labels = torch.argmax(train_probabilities, dim=1).tolist()
#         train_accuracy = accuracy_score(train_labels, train_predicted_labels)
#         train_accuracies.append(train_accuracy)
    
#     # Validation accuracy
#     with torch.no_grad():
#         classifier.eval()
#         val_outputs = classifier(val_embeddings.to(device))
#         val_loss = criterion(val_outputs, val_labels.to(device))
#         val_losses.append(val_loss.item())

#         # Early stopping check
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             counter = 0
#         else:
#             counter += 1

#         val_probabilities = F.softmax(val_outputs, dim=1)
#         val_predicted_labels = torch.argmax(val_probabilities, dim=1).tolist()
#         val_accuracy = accuracy_score(val_labels, val_predicted_labels)

#         # print the current status
#         print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

#         # Early stopping condition
#         if counter >= patience:
#             print("Early Stopping!")
#             break 

In [ ]:
# evaluation on the test set
with torch.no_grad():
    classifier.eval()
    test_outputs = classifier(test_embeddings.to(device))
    test_loss = criterion(test_outputs, test_labels.to(device))

    # convert logits to probs
    test_probabilities = F.softmax(test_outputs, dim=1)

    # get the predicted labels
    test_predicted_labels = torch.argmax(test_probabilities, dim=1).tolist()

    cm = confusion_matrix(test_labels, test_predicted_labels)

    # calculate the accuracy
    test_accuracy = accuracy_score(test_labels, test_predicted_labels)
    print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Plot the confusion matrix
cm = confusion_matrix(labels_test_np, test_predicted_labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


# Plot accuracy and loss
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Validation Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()